In [1]:
## The birth rate
province_list=[]
Filename='province_list.txt'
fid=open(Filename,'r',encoding='utf-8')
for line in iter(fid):
    province_list.append(line.strip())
fid.close()

from bs4 import BeautifulSoup
import urllib
from http import cookiejar
from urllib import request
import requests

import ssl 
context = ssl._create_unverified_context() 
import pandas as pd
import json
import sqlite3
import time
import re

from urllib.parse import quote

con = sqlite3.connect("province.db")

df2 = pd.read_sql("SELECT * FROM url_list", con)

con.commit()


con.close()

b=re.findall(r'\d{6}',str(df2.report))

df2['regcode']=b


header2={
'Accept':'application/json, text/javascript, */*; q=0.01',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8',
'Connection':'keep-alive',
'Host':'data.stats.gov.cn',
'Referer':'',
'Sec-Fetch-Dest':'empty',
'Sec-Fetch-Mode':'cors',
'Sec-Fetch-Site':'same-origin',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}

df3=pd.DataFrame()

for i in range(len(df2)):
	url='https://data.stats.gov.cn/easyquery.htm?cn=E0103&zb=A030201&reg='+str(df2.regcode[i])
	header2['Referer']='https://data.stats.gov.cn/easyquery.htm?cn=E0103&zb=A030201&reg='+str(df2.regcode[i])
	r = requests.get(url,headers=header2,verify=False)
	
	time.sleep(1)
	
	t=time.time()
	t=round(t*1000)
	url2='https://data.stats.gov.cn/easyquery.htm?m=QueryData&dbcode=fsnd&rowcode=reg&colcode=sj&wds=[{"wdcode":"zb","valuecode":"A030201"}]&dfwds=[{"wdcode":"sj","valuecode":"LAST20"}]&k1='+str(t)

	r2=requests.get(url2,headers=header2,cookies=r.cookies,verify=False)
	
	r.connection.close()
	r.close()

	r2.connection.close()
	r2.close()
	
	r_content=r2.json()['returndata']['datanodes']
	r_len=len(r_content)
	
	dfi=pd.DataFrame()
	for j in range(r_len):
		dfi.loc[df2.reg[i],str(r_content[j]['wds'][2]['valuecode'])]=r_content[j]['data']['data']
	
	df3=df3.append(dfi,ignore_index=True)
	
	time.sleep(10)
	
df3=df3.set_index(df2.reg)
df3.head(10)

C:\Users\CAI\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.stats.gov.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\CAI\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.stats.gov.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\CAI\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.stats.gov.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\CAI\anaconda3\lib\site-packages\urllib3\connectionpo

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
reg,,,,,,,,,,,,,,,,,,,,
北京市,0.0,8.24,9.06,9.32,7.96,9.75,8.93,9.05,8.29,7.48,8.06,8.17,8.32,6.26,6.29,6.10,5.10,6.60,6.10,6.20
天津市,0.0,6.67,7.65,7.37,5.84,8.19,8.28,8.75,8.58,8.18,8.30,8.13,7.91,7.67,7.44,7.31,7.14,7.49,7.58,7.72
上海市,0.0,7.20,8.10,9.00,7.52,8.35,8.18,9.56,6.97,7.05,8.64,8.89,9.07,7.47,7.04,6.00,4.85,5.41,5.02,5.30
重庆市,0.0,11.02,11.18,11.77,11.05,10.67,10.37,10.86,9.88,9.17,9.90,10.10,10.10,9.90,9.40,9.45,9.89,9.36,9.70,11.43
河北省,0.0,11.26,13.20,12.42,11.35,13.18,13.04,12.88,13.02,13.22,12.93,13.04,13.33,12.82,12.84,11.98,11.43,11.53,11.16,11.30
山西省,0.0,9.63,11.06,10.29,9.98,10.92,10.81,10.70,10.47,10.68,10.87,11.32,11.30,11.48,12.02,12.36,12.26,12.86,13.06,12.10
内蒙古自治区,0.0,8.35,9.47,9.03,7.72,9.31,8.98,9.17,8.94,9.30,9.57,9.81,10.21,9.87,10.08,9.53,9.24,9.60,10.77,9.90
辽宁省,0.0,6.39,6.49,6.60,6.17,6.49,6.09,6.15,5.71,6.68,6.06,6.32,6.89,6.40,7.01,6.51,6.90,7.38,7.74,10.70
吉林省,0.0,6.62,6.76,5.55,5.87,6.62,5.36,5.73,6.53,7.91,6.69,6.65,7.55,7.67,7.89,7.39,7.25,8.30,8.76,10.31


In [2]:
df2

,reg,data,db,report,sj,zb,regcode
0,北京市,,分省年度数据,cn=E0103&zb=A0P09&reg=110000&sj=2019,2019年,粗离婚率(‰),110000
1,天津市,,分省年度数据,cn=E0103&zb=A0P09&reg=120000&sj=2019,2019年,粗离婚率(‰),120000
2,上海市,,分省年度数据,cn=E0103&zb=A0P09&reg=310000&sj=2019,2019年,粗离婚率(‰),310000
3,重庆市,,分省年度数据,cn=E0103&zb=A0P09&reg=500000&sj=2019,2019年,粗离婚率(‰),500000
4,河北省,,分省年度数据,cn=E0103&zb=A0P09&reg=130000&sj=2019,2019年,粗离婚率(‰),130000
5,山西省,,分省年度数据,cn=E0103&zb=A0P09&reg=140000&sj=2019,2019年,粗离婚率(‰),140000
6,内蒙古自治区,,分省年度数据,cn=E0103&zb=A0P09&reg=150000&sj=2019,2019年,粗离婚率(‰),150000
7,辽宁省,,分省年度数据,cn=E0103&zb=A0P09&reg=210000&sj=2019,2019年,粗离婚率(‰),210000
8,吉林省,,分省年度数据,cn=E0103&zb=A0P09&reg=220000&sj=2019,2019年,粗离婚率(‰),220000
9,黑龙江省,,分省年度数据,cn=E0103&zb=A0P09&reg=230000&sj=2019,2019年,粗离婚率(‰),230000


In [3]:
df3

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
reg,,,,,,,,,,,,,,,,,,,,
北京市,0.0,8.24,9.06,9.32,7.96,9.75,8.93,9.05,8.29,7.48,8.06,8.17,8.32,6.26,6.29,6.10,5.10,6.60,6.10,6.20
天津市,0.0,6.67,7.65,7.37,5.84,8.19,8.28,8.75,8.58,8.18,8.30,8.13,7.91,7.67,7.44,7.31,7.14,7.49,7.58,7.72
上海市,0.0,7.20,8.10,9.00,7.52,8.35,8.18,9.56,6.97,7.05,8.64,8.89,9.07,7.47,7.04,6.00,4.85,5.41,5.02,5.30
重庆市,0.0,11.02,11.18,11.77,11.05,10.67,10.37,10.86,9.88,9.17,9.90,10.10,10.10,9.90,9.40,9.45,9.89,9.36,9.70,11.43
河北省,0.0,11.26,13.20,12.42,11.35,13.18,13.04,12.88,13.02,13.22,12.93,13.04,13.33,12.82,12.84,11.98,11.43,11.53,11.16,11.30
山西省,0.0,9.63,11.06,10.29,9.98,10.92,10.81,10.70,10.47,10.68,10.87,11.32,11.30,11.48,12.02,12.36,12.26,12.86,13.06,12.10
内蒙古自治区,0.0,8.35,9.47,9.03,7.72,9.31,8.98,9.17,8.94,9.30,9.57,9.81,10.21,9.87,10.08,9.53,9.24,9.60,10.77,9.90
辽宁省,0.0,6.39,6.49,6.60,6.17,6.49,6.09,6.15,5.71,6.68,6.06,6.32,6.89,6.40,7.01,6.51,6.90,7.38,7.74,10.70
吉林省,0.0,6.62,6.76,5.55,5.87,6.62,5.36,5.73,6.53,7.91,6.69,6.65,7.55,7.67,7.89,7.39,7.25,8.30,8.76,10.31


In [4]:
con = sqlite3.connect("birth_rate.db")
df3.to_sql('birth', con, if_exists='append')
con.commit()

cur = con.cursor()

result=con.execute("SELECT * FROM birth Limit 5")
for line in result:
    print(line)

con.close()

print(df3.head())

('北京市', 0.0, 8.24, 9.06, 9.32, 7.96, 9.75, 8.93, 9.05, 8.29, 7.48, 8.06, 8.17, 8.32, 6.26, 6.29, 6.1, 5.1, 6.6, 6.1, 6.2)
('天津市', 0.0, 6.67, 7.65, 7.37, 5.84, 8.19, 8.28, 8.75, 8.58, 8.18, 8.3, 8.13, 7.91, 7.67, 7.44, 7.31, 7.14, 7.49, 7.58, 7.72)
('上海市', 0.0, 7.2, 8.1, 9.0, 7.52, 8.35, 8.18, 9.56, 6.97, 7.05, 8.64, 8.89, 9.07, 7.47, 7.04, 6.0, 4.85, 5.41, 5.02, 5.3)
('重庆市', 0.0, 11.02, 11.18, 11.77, 11.05, 10.67, 10.37, 10.86, 9.88, 9.17, 9.9, 10.1, 10.1, 9.9, 9.4, 9.45, 9.89, 9.36, 9.7, 11.43)
('河北省', 0.0, 11.26, 13.2, 12.42, 11.35, 13.18, 13.04, 12.88, 13.02, 13.22, 12.93, 13.04, 13.33, 12.82, 12.84, 11.98, 11.43, 11.53, 11.16, 11.3)
     2019   2018   2017   2016   2015   2014   2013   2012   2011   2010  \
reg                                                                        
北京市   0.0   8.24   9.06   9.32   7.96   9.75   8.93   9.05   8.29   7.48   
天津市   0.0   6.67   7.65   7.37   5.84   8.19   8.28   8.75   8.58   8.18   
上海市   0.0   7.20   8.10   9.00   7.52   8.35   8.18